In [1]:
from athletic_pandas.models import Athlete, WorkoutDataFrame
from goldencheetahlib.client import GoldenCheetahClient
import matplotlib

%matplotlib inline

In [ ]:
gc_client = GoldenCheetahClient('Aart')
athlete = Athlete(cp=300, w_prime=20000)
wdf